In [4]:
import pandas as pd
import numpy as np, matplotlib.pyplot as plt
from pprint import pprint
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing 
from sklearn.metrics import accuracy_score, mean_squared_error, confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV
import pickle
from keras import Sequential
import tensorflow as tf
#from keras import InputLayer
from collections import deque
import random
import keras

In [32]:
with open('clf0.pickle', 'rb') as handle:
    clf0 = pickle.load(handle)

    
with open('clf1.pickle', 'rb') as handle:
    clf1 = pickle.load(handle)
    

with open('clf2.pickle', 'rb') as handle:
    clf2 = pickle.load(handle)
    


    
model = keras.models.load_model('q_network2')



In [33]:
test=pd.read_csv('C:/Users/Caliméromaric/Downloads/test.csv',  sep=';' , decimal=',') 

test['G'] = [x.replace(',', '.') for x in test['G']]

test['G'] =test['G'].astype(float)

df_test= test.copy()
df_test[['AA', 'B'  ] ] = df_test[['AA', 'B'  ] ].apply(preprocessing.LabelEncoder().fit_transform)
df_test= df_test.drop(columns=[ 'Day'])
test [ 'Strategy'] =0
test [ 'Success'] =0

In this work , we solve a reinforcement learning case study for Axa. We are asked to implement a Q-learning algorithm.
The objective is to find a suitable strategy per customer contract to maximize Gain and minimize Cost according to the characteristics of the customer.

In this part,   We  evaluate the model  on the test data 

. We choose the strategy that maximise Q at each state. We simulate the sequential game with the probabilities of success by strategy.
We compute the expected  total net gain with the probabilities of success: 
    $\sum p*Profit + (1-p)*Loss$

    
We also observe the realization of the total net gain for the simulation 

In [34]:

globat_net_gain = 0 
expected_gain= 0

for i in range(len(df_test.A)):
    
            df_test.iloc[i].Strategy   = 0 
        
            #Observe current state
            row= df_test.iloc[i] 
            
            X= np.array(row).reshape(1, -1)
            
            probs = [  clf0.predict_proba(X)[0,1], clf1.predict_proba(X)[0,1], clf2.predict_proba(X) [0,1]]
            current_state =  np.array([globat_net_gain, 
                             probs[0], probs[1], probs[2], row.Gain, row.Cost]).reshape(1, -1)
            
            action = np.argmax(model.predict(current_state))
            test.at[  i, 'Strategy']    = action
         
            #Simulate customer reaction to strategy action

            prob =probs[action]

            success_ind  = 1 if np.random.random() < prob else 0    

            reward = success_ind*(row.Gain-row.Cost) + (1-success_ind)*(-5)

            globat_net_gain += reward/ len(test.Strategy)
            
            expected_gain += prob*(row.Gain-row.Cost) + (1-prob)*(-5)
            
            test.at[  i, 'Success']    = success_ind
            #print(i)

In [35]:
#the realization of the total net gain for the simulation 
globat_net_gain*len(test.Strategy)

14317.99999999961

In [36]:
#Rate of success
np.mean(test [ 'Success'] )

0.556

This rate of  success is much better than the initial 30% that we observed in the train data

In [37]:
#sum p*Profit + (1-p)*Loss
expected_gain

14016.408798841234

In [38]:
# Saving result 
test.to_csv(r'final_result.csv')

In [30]:
test

,index,Day,A,B,C,D,E,F,G,H,...,W,X,Y,Z,AA,BB,Gain,Cost,Strategy,Success
0,0,4,0,book,0,2,37,5,20.0,16,...,1,-6.233518,11,4,pomme,0.65,174.0,150.0,2,0
1,1,4,0,book,0,4,29,2,10.0,7,...,5,-6.151616,28,0,poire,0.20,74.0,50.0,0,0
2,2,4,0,book,0,4,32,4,10.0,6,...,1,-6.234130,14,12,peche,0.59,59.0,51.0,2,1
3,3,4,0,book,0,4,27,1,5.0,4,...,2,-6.338930,2,0,pomme,0.39,94.0,71.0,2,1
4,4,4,0,book,0,4,31,3,10.0,5,...,1,-6.326898,2,9,pomme,0.49,87.0,71.0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1995,4,0,book,0,4,33,4,10.0,8,...,1,-6.220889,14,8,peche,0.54,74.0,57.0,1,1
1996,1996,4,0,book,0,4,32,5,10.0,8,...,2,-6.173538,16,6,prune,0.24,120.0,80.0,1,1
1997,1997,4,0,book,0,4,33,4,5.0,9,...,5,-6.176252,15,2,poire,0.81,102.0,63.0,1,1
1998,1998,4,0,book,0,4,32,4,10.0,7,...,1,-6.273317,0,10,peche,0.09,57.0,49.0,1,0
